In [ ]:
pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 10.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

tqdm.pandas()

# Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# Load models
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

roberta_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
roberta_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

vader = SentimentIntensityAnalyzer()

# Scoring function
def run_model_batched(texts, tokenizer, model, label_map, batch_size=32):
    model.eval()
    model.to(device)
    all_preds = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Batches"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors='pt', truncation=True, padding=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1).tolist()
            scaled = [label_map[p] for p in preds]
            all_preds.extend(scaled)

    return all_preds

def vader_scaled(text):
    score = vader.polarity_scores(text)['compound']
    if score >= 0.05:
        return 5.0
    elif score <= -0.05:
        return 0.0
    else:
        return 2.5


Using: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Setup for model outputs
finbert_map = {0: 0.0, 1: 2.5, 2: 5.0}
roberta_map = {0: 0.0, 1: 2.5, 2: 5.0}

stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA"]
for stock in stocks:
    print(f"\n 🚀 Processing {stock}...")

    # Load
    df = pd.read_csv(f"{stock}_finbert_input.csv")

    # VADER
    print("→ Running VADER...")
    df["sentiment_vader"] = df["string_input"].progress_apply(vader_scaled)

    # FinBERT
    print("→ Running FinBERT...")
    df["sentiment_finbert"] = run_model_batched(df["string_input"].tolist(), finbert_tokenizer, finbert_model, finbert_map)

    # RoBERTa
    print("→ Running RoBERTa...")
    df["sentiment_roberta"] = run_model_batched(df["string_input"].tolist(), roberta_tokenizer, roberta_model, roberta_map)

    # Save per stock
    df_out = df[["index", "Date", "sentiment_vader", "sentiment_finbert", "sentiment_roberta"]]
    df_out.to_csv(f"{stock}_sentiment_output.csv", index=False)
    print(f"💾 Saved {stock}_sentiment_output.csv")



🚀 Processing AAPL...
→ Running VADER...


100%|██████████| 104345/104345 [09:16<00:00, 187.36it/s]


→ Running FinBERT...


Batches: 100%|██████████| 3261/3261 [04:45<00:00, 11.43it/s]


→ Running RoBERTa...


...

→ Running FinBERT...


Batches: 100%|██████████| 1282/1282 [02:53<00:00,  7.39it/s]


→ Running RoBERTa...


Batches: 100%|██████████| 1282/1282 [02:51<00:00,  7.48it/s]

💾 Saved TSLA_sentiment_output.csv


In [ ]:
from google.colab import files
files.download("AAPL_sentiment_output.csv")
files.download("MSFT_sentiment_output.csv")
files.download("AMZN_sentiment_output.csv")
files.download("GOOGL_sentiment_output.csv")
files.download("META_sentiment_output.csv")
files.download("NVDA_sentiment_output.csv")
files.download("TSLA_sentiment_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>